# 3. Modelo C
Basado en el conocimiento del "negocio".\
Según las fuentes consultadas, el sobrepeso se calcula en base al IMC, el cual a su vez se calcula diviendo el peso en kg entre el cuadrado de la altura en metros.

<center>

$IMC = kg/m^2$ 



| IMC        	| Diagnóstico       	|
|------------	|-------------------	|
| <18.5      	| Peso insuficiente 	|
| (18.5, 25) 	| Peso normal       	|
| (25, 30)   	| Sobrepeso         	|
| >30        	| Obesidad          	|

</center>

\
El principal factor adicional que interviene en esta categorización es si el sujeto es deportista o no, y más concretamente si se dedica a la alterofilia. Un sujeto con estas características puede dar positivo en sobrepeso pero no tener nada de masa grasa, debido a su masa muscular. Por desgracia, el dataset ofrece este último dato de manera poco exhaustiva: la FAF sólo tiene valores enteros del 0 al 2. \
\
Procedimiento:
- Crear un modelo con las variables peso y altura
- Crear un segundo modelo añadiendo la variable faf
- Comparar los modelos entre sí

## Librerías

In [ ]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Modelos
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score, \
roc_curve, roc_auc_score, ConfusionMatrixDisplay, multilabel_confusion_matrix

# Otros
import warnings
warnings.filterwarnings('ignore')

## Carga de datos

In [ ]:
df = pd.read_csv(r'..\data\processed\train_3.csv')
df_1 = df[['height', 'weight', 'nobeyesdad']]
df_2 = df[['height', 'weight', 'faf', 'nobeyesdad']]

## División de datos

In [ ]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(df_1.drop(columns=['nobeyesdad']),
                                                    df_1['nobeyesdad'],
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=df_1['nobeyesdad'])

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df_2.drop(columns=['nobeyesdad']),
                                                    df_2['nobeyesdad'],
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=df_2['nobeyesdad'])

## Baselines

In [ ]:
modelos = {LogisticRegression(random_state=42):'LogR',
           SVC(kernel='linear', random_state=42): 'SVC_linear',
           SVC(kernel='poly', degree=4): 'SVC_poly',
           SVC(random_state=42):'SVC_rbf',
           DecisionTreeClassifier(random_state=42):'DT',
           RandomForestClassifier(random_state=42, class_weight='balanced'):'RF',
           KNeighborsClassifier(n_neighbors=5):'KNEIGH',
           lgb.LGBMClassifier():'LGBM',
           XGBClassifier():'XGB'}

scores = ['accuracy']
data_1 = []
data_2 = []

for modelo in modelos:
    print(f'processing {modelo}')
    data_1.append([(cross_val_score(modelo, X_train_1, y_train_1, cv=5, scoring=score)).mean() for score in scores])
    data_2.append([(cross_val_score(modelo, X_train_2, y_train_2, cv=5, scoring=score)).mean() for score in scores])

baselines = pd.DataFrame([data_1, data_2], columns=['accuracy', 'accuracy (faf)'], index=modelos.values())
baselines.sort_values(by='accuracy', ascending=False)

## Entrenamiento y validación

In [ ]:
MODELO_ELEGIDO.fit()
accuracy_score(y_test, MODELO_ELEGIDO.predict(X_test))

## Guardado

In [ ]:
import joblib

filename = 'my_model_3.sav'
joblib.dump(MODELO_ELEGIDO, filename)